Fully connected Neural Network using Pytorch


*   Dataset: MNIST

*   Network type: Fully connected dense neural network



Import the necessary packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Create a Fully connected Neural Network

In [2]:
class NN (nn.Module):
  '''
  Simple 2 layer Neural Network
  Input: input_size [int]
         num_classes
  Output: vector [size= batch size x num_classes]. Each entry is a logit score.
  '''
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__()
    self.fc1=nn.Linear(input_size,64)
    self.fc2=nn.Linear(64,128)
    self.fc3=nn.Linear(128,10)

  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x




Test the model on the random input

In [3]:
model= NN(784,10)
xx=torch.rand((64,784))
output=model(xx)
print(output.shape)

torch.Size([64, 10])


Set the device

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [5]:
input_size=784 # MNIST dataset 28x28 image size
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=1

Load the data

In [6]:
train_data= datasets.MNIST(root='/dataset',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)

test_data=datasets.MNIST(root='/dataset',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_data,batch_size=64, shuffle=True)

Initialize the Network

In [7]:
model=NN(input_size=input_size,num_classes=num_classes).to(device)

Loss and Optimizer

In [8]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

Training the Network

In [9]:
for epoch in range(num_epochs):
  for batch_idx, (data,targets) in enumerate(train_loader):
    # Get data to GPU if available
    data=data.to(device=device)
    targets=targets.to(device=device)
    # Flatten the image data
    data=data.reshape(data.shape[0],-1)
    #pass the data through model
    predictions=model(data)
    loss=criterion(predictions,targets)
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    # gradient descent or Adam step
    optimizer.step()


Check the accuracy of the model

In [10]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print('Training accuracy')
  else:
    print('Test accuracy')

  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      x=x.reshape(x.shape[0],-1)
      scores=model(x)
      _,predictions=scores.max(1)

      num_correct+=(predictions==y).sum()
      num_samples+=predictions.shape[0]
      acc=(float(num_correct)/float(num_samples)) *100
    print(f'Got {num_correct}/{num_samples} with accuracy {(float(num_correct)/float(num_samples)) *100 :.2f} %')
  
  model.train()

In [11]:
check_accuracy(test_loader,model)
check_accuracy(train_loader,model)

Test accuracy
Got 9469/10000 with accuracy 94.69 %
Training accuracy
Got 56924/60000 with accuracy 94.87 %
